<a href="https://colab.research.google.com/github/Breno-Giacoppini/Challange-4-DISRUPTIVE-ARCHITECTURES-IOT-IOB-GENERATIVE-IA/blob/main/sistema_de_previsao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Função para importar dados de um arquivo CSV
def importar_dados_csv(caminho_csv):
    """
    Importa dados de um arquivo CSV e retorna um DataFrame.

    Args:
    caminho_csv (str): Caminho para o arquivo CSV.

    Returns:
    pd.DataFrame: DataFrame contendo os dados importados.
    """
    try:
        dados_csv = pd.read_csv(caminho_csv, encoding='utf-8', sep=';')
        print("Arquivo CSV carregado com sucesso!")
        return dados_csv
    except FileNotFoundError:
        print("Erro: Arquivo CSV não encontrado.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocorreu um erro ao carregar o arquivo CSV: {e}")
        return pd.DataFrame()

# Função para coletar dados manualmente
def coleta_de_dados():
    """
    Coleta dados de vendas manualmente, incluindo modelo, tamanho e quantidade por período.

    Returns:
    pd.DataFrame: DataFrame contendo os dados de vendas coletados.
    """
    vendas = []
    while True:
        # Escolha do modelo
        modelo = input("\nSelecione o modelo (Camiseta, Calça, Jaqueta, Bermuda ou outro): ")
        tamanho = input("Informe o tamanho (PP, P, M, G, GG): ").upper()

        # Validação dos dados
        if not modelo or not tamanho:
            print("Modelo ou tamanho inválido, tente novamente.")
            continue

        # Coleta dos períodos de vendas
        periodo_atual = 1
        while True:
            try:
                quantidade = int(input(f"\nInforme a quantidade de vendas para o período {periodo_atual} de {modelo} (tamanho {tamanho}): "))
                vendas.append({'modelo': modelo, 'tamanho': tamanho, 'quantidade': quantidade, 'periodo': periodo_atual})
                periodo_atual += 1
                continuar = input("Deseja informar mais um período? (s/n): ").lower()
                if continuar != 's':
                    break
            except ValueError:
                print("Quantidade inválida. Por favor, insira um número.")

        # Pergunta se deseja adicionar outro produto
        adicionar_produto = input("\nDeseja cadastrar outro produto? (s/n): ").lower()
        if adicionar_produto != 's':
            break

    return pd.DataFrame(vendas)

# Função para treinar e avaliar o modelo de previsão
def treinar_modelo(dados):
    """
    Treina um modelo de Random Forest para prever as quantidades vendidas e avalia a precisão do modelo.

    Args:
    dados (pd.DataFrame): DataFrame contendo os dados de vendas.

    Returns:
    tuple: (modelo treinado, encoder, avaliação do modelo)
    """
    # Codificação das variáveis categóricas
    encoder = OneHotEncoder(handle_unknown='ignore')
    X = encoder.fit_transform(dados[['modelo', 'tamanho', 'periodo']])
    y = dados['quantidade']

    # Divisão dos dados
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    modelo_ml = RandomForestRegressor(random_state=42)
    modelo_ml.fit(X_train, y_train)

    # Avaliação do modelo
    y_pred = modelo_ml.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("\nAvaliação do modelo:")
    print(f"Erro Médio Absoluto (MAE): {mae}")
    print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")

    return modelo_ml, encoder

# Função para prever vendas para o próximo período
def previsao_proximo_periodo(dados, modelo, encoder):
    """
    Gera previsões de vendas para o próximo período com base no modelo treinado.

    Args:
    dados (pd.DataFrame): DataFrame com os dados de vendas.
    modelo: Modelo de RandomForestRegressor treinado.
    encoder: OneHotEncoder usado para transformar dados categóricos.

    Returns:
    pd.DataFrame: DataFrame contendo as previsões de vendas.
    """
    previsoes = []
    for (modelo_nome, tamanho), group in dados.groupby(['modelo', 'tamanho']):
        ultimo_periodo = group['periodo'].max()
        proximo_periodo = pd.DataFrame({
            'modelo': [modelo_nome],
            'tamanho': [tamanho],
            'periodo': [ultimo_periodo + 1]
        })
        X_novo_periodo = encoder.transform(proximo_periodo[['modelo', 'tamanho', 'periodo']])
        predicao = modelo.predict(X_novo_periodo)[0]
        previsoes.append({'modelo': modelo_nome, 'tamanho': tamanho, 'quantidade_predita': predicao})

    previsoes_df = pd.DataFrame(previsoes)
    return previsoes_df

# Função principal
def main():
    """
    Função principal que executa a coleta de dados, treinamento do modelo e previsões de vendas.
    """
    opcao = input("Deseja inserir os dados manualmente ou importar um arquivo CSV?\n1. Inserir manualmente\n2. Importar CSV\nDigite o número da opção escolhida: ")

    if opcao == "2":
        caminho_csv = input("Por favor, informe o caminho do arquivo CSV: ")
        dados_vendas = importar_dados_csv(caminho_csv)
    else:
        dados_vendas = coleta_de_dados()

    if dados_vendas.empty or len(dados_vendas['periodo'].unique()) < 2:
        print("Erro: Dados insuficientes para treinar o modelo.")
        return

    modelo_ml, encoder = treinar_modelo(dados_vendas)
    previsoes_df = previsao_proximo_periodo(dados_vendas, modelo_ml, encoder)

    # Exibir previsões
    print("\nPrevisão de vendas para o próximo período:")
    for _, row in previsoes_df.iterrows():
        print(f"- {round(row['quantidade_predita'])} unidades de {row['modelo']} (tamanho {row['tamanho']}) no próximo período.")

    # Relatório detalhado (opcional)
    print("\nRelatório de Vendas:")
    print(dados_vendas.groupby(['modelo', 'tamanho', 'periodo']).sum().reset_index())

if __name__ == "__main__":
    main()
